# Dolo

[Installation instructions](https://github.com/EconForge/dolo/wiki/Installation) for dolo involve a number of dependencies, including the dolo language "dolang."  This notebook assumes all these have been installed.

In [1]:
from dolo import *
import BufferStockParameters

In [2]:
import sys
import os
os.getcwd()

'/home/normann/Dropbox/Pablo/DARKolo/chimeras/BufferStock'

In [3]:
model_dolo = yaml_import("./bufferstock.yaml")
print( model_dolo )


        Model:
        ------
        name: "None"
        type: "dtcc"
        file: "<string>

Equations:
----------

transition
 1  : 1.7554 : m[t] = exp(tran[t]) + (m[t-1] - (c[t-1]))*((R)/((Γ)*(exp(perm[t]))))

arbitrage
 1  : -0.8726 : ((R)*(β))*(((((c[t+1])*(exp(perm[t+1])))*(Γ))/(c[t]))^(-(ρ))) - (1)

definitions





## Explaining the Dolo Model File 

Each item in the `bufferstock.yaml` file corresponds to an equation in the formal mathematics of the BufferStockTheory problem.

### arbitrage:
called the `no-arbitrage` equation since the idea is that the amount by which you could make yourself better is zero).  

See the dolo/dolark documentation to absorb syntactic conventions like the fact that time offsets are signified by parens; e.g., `c[t-1]` is the variable `c` one period in the past.

If the model is stochastic, equations that involve a future-dated variable are calculated in expectation.

Dolo's specification of the model defines `perm` as the logarithm of what is called $\psi$, so its exponential `exp(perm[t+1])` is equivalent to $\psi_{t+1}$ in the companion notebook's notation.  Finally, the expression `| 0.0 <= c[t] <= m[t]` corresponds to the liquidity constraint mentioned above (mathematically, $0 \leq c \leq m$). Thus, the arbitrage equation:

    arbitrage:
        - (R*β*((c[t+1]*exp(perm[t+1])*Γ)/c[t])^(-ρ)-1 ) | 0.0<=c[t]<=m[t]

is equivalent to the Euler equation in the HARK notebook, because:

\begin{align*}
0 & = R \beta \mathbb{E}_{t}[(\psi c_{t+1}\Gamma /c_{t})^{-\rho})]-1 \\ 
1 & = R \beta \mathbb{E}_{t}[(\Gamma \psi \frac{c_{t+1}}{c_{t}})^{-\rho})] 
\\c_{t}^{-\rho} & = R \beta \mathbb{E}_{t}[(\Gamma \psi c_{t+1})^{-\rho})] \\
\end{align*}

### transition:

The second equation in the `dolang` model is:

    transition:
        - m[t] = exp(tran[t]) + (m[t-1]-c[t-1])*(R/(Γ*exp(perm[t])))

Recall that the dynamic budget constraints in the HARK problem are:
\begin{eqnarray*}
a_t &=& m_t - c_t \\
m_{t+1} &=& R/(\Gamma \psi_{t+1}) a_t + \theta_{t+1} \\
\end{eqnarray*}

Substituting the first condition, defining $a_t$, into the second, and rearranging, we get:
\begin{eqnarray}
m_{t+1} &=& \theta_{t+1} +  (m_t - c_t) R/(\Gamma \psi_{t+1})
\end{eqnarray}
and $\theta_{t+1}$ is equivalent to the exponential of the lognormally-distributed stochastic variable `tran`.  (In dolo, the equation's timing is shifted back a period because its expectation is not being taken)

In [4]:
max_m = BufferStockParameters.Common['max_m']
model_dolo.data['calibration']['max_m'] = max_m

Obtain the decision rule by time iteration amd improved time iteration

In [5]:
dr = time_iteration(model_dolo, maxit = 5, verbose=True)
dr = improved_time_iteration(model_dolo, tol=1e-08, dr0=dr, verbose=True)
# %%

------------------------------------------------
| N   |  Error     | Gain     | Time     | nit |
------------------------------------------------
|   1 |  9.000e-01 |      nan |    2.523 |   5 |
|   2 |  2.382e-02 |    0.026 |    0.047 |   3 |
|   3 |  2.409e-02 |    1.011 |    0.051 |   3 |
|   4 |  2.436e-02 |    1.011 |    0.045 |   3 |
|   5 |  2.463e-02 |    1.011 |    0.048 |   3 |
------------------------------------------------
Elapsed: 2.713557004928589 seconds.
------------------------------------------------
------------------------------------------------------------------------------------------------------
| Start Improved Time Iterations.                                                                    |
------------------------------------------------------------------------------------------------------
| N    | f_x       | d_x       | Time_residuals | Time_inversion | Time_search | Lambda_0  | N_invert | N_search |
--------------------------------------------------